In [1]:
using DataFrames, CSV
#using JLD
using JuMP, Gurobi
using LinearAlgebra, Random, Printf, StatsBase, CategoricalArrays
using Plots, StatsPlots
using Distributions

In [2]:
races = ["Sakhir", "Jeddah", "Melbourne", "Shanghai", "Baku", "Miami", "Imola", "Monaco", "Barcelona", "Montreal", "Spielberg", "Silverstone", "Budapest", "Spa", "Zandvoort", "Monza", "Singapore", "Suzuka", "Lusail", "Austin", "Mexico City", "Sao Paulo", "Las Vegas","Yas Marina"]
races_circuitref = ["bahrain", "jeddah", "albert_park", "shanghai", "baku", "miami", "imola", "monaco", "catalunya", "villeneuve", "red_bull_ring", "silverstone", "hungaroring", "spa", "zandvoort", "monza", "marina_bay", "suzuka", "losail", "americas", "rodriguez", "interlagos", "las_vegas", "yas_marina"];

In [3]:
#import Data
all_df = CSV.read("circuits.csv", DataFrame)
first(all_df,5)

,circuitId,circuitRef,name,location,country
,Int64,String15,String,String31,String15
1,1,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia
2,2,sepang,Sepang International Circuit,Kuala Lumpur,Malaysia
3,3,bahrain,Bahrain International Circuit,Sakhir,Bahrain
4,4,catalunya,Circuit de Barcelona-Catalunya,Montmeló,Spain
5,5,istanbul,Istanbul Park,Istanbul,Turkey


In [4]:
select!(all_df, "circuitRef", "location", "lat", "lng")

#only keep rows for the races that we care about, i.e. 2024 schedule
all_df[in(races_circuitref).(all_df.circuitRef), :]

,circuitRef,location,lat,lng
,String15,String31,Float64,Float64
1,albert_park,Melbourne,-37.8497,144.968
2,bahrain,Sakhir,26.0325,50.5106
3,catalunya,Montmeló,41.57,2.26111
4,monaco,Monte-Carlo,43.7347,7.42056
5,villeneuve,Montreal,45.5,-73.5228
6,silverstone,Silverstone,52.0786,-1.01694
7,hungaroring,Budapest,47.5789,19.2486
8,spa,Spa,50.4372,5.97139
9,monza,Monza,45.6156,9.28111


In [5]:
using Geodesy

# Gives distance between two circuits in km
function dist(c1, c2)
    circuit1 = all_df[c1, :]
    circuit2 = all_df[c2, :]
    lat1 = circuit1[:lat]
    lng1 = circuit1[:lng]
    lat2 = circuit2[:lat]
    lng2 = circuit2[:lng]
    return euclidean_distance(LLA(lat1, lng1, 0), LLA(lat2, lng2, 0)) / 1000
end

dist(1,12)

12386.36377883742

In [6]:
#create a matrix of distances between each circuit
distance_matrix=[dist(i,j) for i in 1:24, j in 1:24]

24×24 Matrix{Float64}:
     0.0    6067.92  10361.5    12339.2    …   7556.33   10100.7    10082.2
  6067.92      0.0    5818.24    9417.92       4922.78    5418.79   12100.1
 10361.5    5818.24      0.0     4611.4        7542.94     447.152  10994.7
 12339.2    9417.92   4611.4        0.0        9252.61    5017.04    9327.96
 11597.7    7749.32   2536.63    2254.5        8118.92    2941.47   10454.3
 12232.2    9109.4    4255.09     486.382  …   8937.29    4658.66    9644.26
 12317.0   11632.9    9204.84    5699.13       9431.36    9461.91    8271.26
 12320.5    9310.67   4630.56     592.452      8888.76    5021.37    9631.6
 12365.8    9443.78   5024.18    1192.61       8671.2     5392.55    9750.33
 12206.6    9011.25   4339.68     991.471      8548.98    4721.25    9940.26
     ⋮                                     ⋱                        
  7584.88   5116.5    7690.07    9296.91   …    225.276   7480.79   12643.6
  7518.32   3711.89   6497.96    8867.44       1477.14    6227.66  

In [7]:
function powerset(x::AbstractSet)   
    result = Set([Set()])
    for  elem in x, j in result
        union!(result, Set([union(j,elem)]))
    end
    delete!(result, Set())
    result
end

powerset (generic function with 1 method)

In [8]:
races = 15
K = 6;

In [10]:
V = Set(1:races)
V_0 = setdiff(V, [1])
all_S = powerset(V_0);

In [17]:
for S in all_S
    if length(S) < 2
        delete!(all_S, S)
    end
    if length(S) > length(V) - 2
        delete!(all_S, S)
    end
    if S == V_0
        delete!(all_S, S)
    end
end

In [18]:
model = Model(Gurobi.Optimizer)
set_optimizer_attribute(model, "OutputFlag", 1)

# variables
@variable(model, x[1:races, 1:races], Bin)

# constraints
@constraint(model, only_one_in[j in V_0], sum(x[i, j] for i in V) == 1)
@constraint(model, only_one_out[i in V_0], sum(x[i, j] for j in V) == 1) 
@constraint(model, no_self_connect[i in V], x[i, i] == 0)
@constraint(model, K_petals_in, sum(x[i, 1] for i in V_0) == K)
@constraint(model, K_petals_out, sum(x[1, j] for j in V_0) == K)

for S in all_S
    @constraint(model, sum(x[i, j] for i in S, j in S) <= length(S) - 1)
end

#objective
@objective(model, Min, sum(sum(x[i, j] * distance_matrix[i, j] for i in V) for j in V));

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-17


In [19]:
#latex_formulation(model)

In [20]:
optimize!(model)

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads
Optimize a model with 15088 rows, 225 columns and 775672 nonzeros
Model fingerprint: 0xc9ec05f4
Variable types: 0 continuous, 225 integer (225 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+02, 1e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+01]
Found heuristic solution: objective 162973.08636
Presolve removed 15 rows and 15 columns
Presolve time: 0.76s
Presolved: 15073 rows, 210 columns, 671330 nonzeros
Variable types: 0 continuous, 210 integer (210 binary)

Root relaxation: objective 1.204435e+05, 47 iterations, 0.22 seconds (0.34 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    120443.51093 120443.511  0.00%     -    1s

Explored 1 nodes (47 s

In [21]:
objective_value(model)

120443.51093402033

In [22]:
x_opt = value.(x)

15×15 Matrix{Float64}:
  0.0   1.0   1.0  -0.0   1.0  -0.0  …  -0.0   1.0   0.0  -0.0  -0.0   1.0
  1.0   0.0  -0.0  -0.0  -0.0  -0.0     -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
  1.0  -0.0   0.0  -0.0  -0.0  -0.0     -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0   0.0  -0.0   0.0     -0.0  -0.0   1.0  -0.0  -0.0  -0.0
  1.0  -0.0  -0.0  -0.0   0.0  -0.0     -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0   1.0  -0.0   0.0  …  -0.0  -0.0  -0.0  -0.0   0.0  -0.0
  1.0  -0.0  -0.0  -0.0  -0.0  -0.0     -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0   0.0  -0.0  -0.0      0.0  -0.0  -0.0   0.0   1.0  -0.0
  0.0  -0.0  -0.0  -0.0  -0.0  -0.0     -0.0  -0.0  -0.0   0.0  -0.0  -0.0
 -0.0  -0.0  -0.0  -0.0  -0.0  -0.0      0.0  -0.0  -0.0   1.0  -0.0  -0.0
  0.0  -0.0  -0.0  -0.0  -0.0  -0.0  …   1.0   0.0  -0.0  -0.0   0.0  -0.0
  1.0  -0.0  -0.0   0.0  -0.0  -0.0     -0.0  -0.0   0.0  -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0  -0.0  -0.0  -0.0      0.0  -0.0  -0.0   0.0  -0.0  -0.0
 -